In [2]:
import numpy as np
inputs = np.array([
    ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"],
    ["Z","Y","X","W","V","U","T","S","R","Q","P","O","N","M","L","K","J","I","H","G","F","E","D","C","B","A"],
    ["B","D","F","H","J","L","N","P","R","T","V","X","Z","A","C","E","G","I","K","M","O","Q","S","U","W","Y"],
    ["M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A","B","C","D","E","F","G","H","I","J","K","L"],
    ["H","G","F","E","D","C","B","A","L","K","J","I","P","O","N","M","U","T","S","R","Q","X","W","V","Z","Y"]
])

expected = np.array([
    ["B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A"],
    ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"],
    ["C","E","G","I","K","M","O","Q","S","U","W","Y","A","B","D","F","H","J","L","N","P","R","T","V","X","Z"], 
    ["N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A","B","C","D","E","F","G","H","I","J","K","L","M"],
    ["I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A","B","C","D","E","F","G","H"]
])

In [3]:
import string

# utilty fn that converts the list of strings into a list of one hot encoded vectors
def string_to_one_hot(inputs: np.ndarray) -> np.ndarray:
    char_to_index = {char: i for i, char in enumerate(string.ascii_uppercase)}

    one_hot_inputs = []
    for row in inputs:
        one_hot_list = []
        for char in row:
            if char.upper() in char_to_index:
                one_hot_vector = np.zeros((len(string.ascii_uppercase), 1))
                one_hot_vector[char_to_index[char.upper()]] = 1
                one_hot_list.append(one_hot_vector)
        one_hot_inputs.append(one_hot_list)

    return np.array(one_hot_inputs)

In [4]:
class InputLayer:
    def __init__(self, inputs, hidden_size):
        self.inputs_x = inputs
        self.input_weights = np.random.uniform(low = 0, high = 1, size=(hidden_size,len(inputs[0])))
        self.input_grad = np.zeros_like(self.input_weights)

    def get_input(self,timestep):
        return self.inputs_x[timestep]
    
    def weighted_sum(self,timestep):
        return self.input_weights @ self.get_input(timestep)
    
    def gradient_per_timestep(self, time_step, weighted_sum_grad):
        self.input_weights += weighted_sum_grad @ self.get_input(time_step).T

    def weights_update(self,learning_rate):
        self.input_weights -= learning_rate * self.input_grad
        

In [5]:
class HiddenLayer:
    def __init__(self,vocab_size,size):
        self.hidden_weights = np.random.uniform(low=0, high=1, size=(size,size))
        self.bias_b = np.random.uniform(low=0,high=1,size=(size,1)) 
        self.states = np.zeros(shape=(vocab_size, size, 1))
        self.next_activation_grad = np.zeros(shape=(size,1))
        self.bias_grad = np.zeros_like(self.bias_b)
        self.weights_gradient = np.zeros_like(self.hidden_weights)

    def get_hidden_state(self,timestep):
        if timestep < 0:
            return np.zeros_like(self.states[0])
        else:
            return self.states[timestep]

    def set_hidden_state(self,timestep,hidden_state):
        self.states[timestep] = hidden_state

    def activate(self, weighted_input,timestep):
        prev_hidden_state = self.get_hidden_state(timestep-1)
        weighted_hidden_state = self.hidden_weights @ prev_hidden_state    
        weighted_sum = weighted_input + weighted_hidden_state + self.bias_b    
        activation = np.tanh(weighted_sum)
        self.set_hidden_state(timestep,activation)
        return activation
    
    def calculate_gradients_per_step(self, timestep, output_grad):
        activation_grad = output_grad + self.next_activation_grad
        weighted_sum_grad = activation_grad * (1 - self.get_hidden_state(timestep) ** 2)

        self.next_activation_grad = self.hidden_weights.T @ weighted_sum_grad

        self.weights_gradient += weighted_sum_grad @ self.get_hidden_state(timestep -1).T

        self.bias_grad += weighted_sum_grad 

        return weighted_sum_grad
    
    def update_parameters(self, learninig_rate):
        self.hidden_weights -= learninig_rate * self.weights_gradient
        self.bias_b -= learninig_rate * self.bias_grad 



In [6]:
from scipy.special import softmax

class OutputLayer:
    def __init__(self,size,hidden_size):
        self.output_weights = np.random.uniform(low=0, high=1, size=(size,hidden_size))
        self.bias = np.random.uniform(low=0,high=1,size=(size,1))
        self.states = np.zeros(shape=(size,size,1))
        self.grad_bias = np.zeros_like(self.bias)
        self.grad_output_weights = np.zeros_like(self.output_weights)

    def predict(self,hidden_state,timestep):
        output = self.output_weights @ hidden_state + self.bias
        prediction = softmax(output, axis=0)
        self.set_state(timestep,prediction)
        return prediction

    def set_state(self,timestep,prediction):
        self.states[timestep] = prediction

    def get_state(self,timestep):
        return self.states[timestep]
    
    def calculate_grad_per_step(self,expected,hidden_state, timestep):
          # dL_do = dL_dyhat * dyhat_do = derivative of loss function * derivative of softmax
        # dL_do = step.y_hat - expected[step_number]

        output_grad = self.get_state(timestep) - expected
        
        # (input_size, 1) @ (1, hidden_size) = (input_size, hidden_size)
        self.grad_output_weights  += output_grad @ hidden_state.T

        self.grad_bias += output_grad
        return self.output_weights.T @ output_grad
        
    def update_parameters(self,learning_rate):
        self.output_weights -= learning_rate * self.grad_output_weights
        self.bias -= learning_rate * self.grad_bias

        


In [7]:
class MyRNN:
    def __init__(self,vocab_size,hidden_size,learning_rate):
        self.hidden_layer = HiddenLayer(vocab_size,hidden_size)
        self.output_layer = OutputLayer(vocab_size,hidden_size)
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

    def feedfordward(self,inputs):
        self.input_layer = InputLayer(inputs,self.hidden_size)
        for step in range(len(inputs)):
            weighted_input= self.input_layer.weighted_sum(step)
            activation = self.hidden_layer.activate(weighted_input,step)
            self.output_layer.predict(activation,step)
        return self.output_layer
    
    def backpropagation(self,expected):
        for step in reversed(range(len(expected))):
            output_grad = self.output_layer.calculate_grad_per_step(expected[step],self.hidden_layer.get_hidden_state(step),step)
            weighted_sum_grad = self.hidden_layer.calculate_gradients_per_step(step,output_grad)
            self.input_layer.gradient_per_timestep(step,weighted_sum_grad)

        self.output_layer.update_parameters(self.learning_rate)
        self.hidden_layer.update_parameters(self.learning_rate)
        self.input_layer.weights_update(self.learning_rate)

    def loss(self, y_hat , y) :
            """
            Cross-entropy loss function - Calculating difference between 2 probability distributions.
            First, calculate cross-entropy loss for each time step with np.sum, which returns a numpy array
            Then, sum across individual losses of all time steps with sum() to get a scalar value.
            :param y_hat: predicted value
            :param y: expected value - true label
            :return: total loss
            """
            return sum(-np.sum(y[i] * np.log(y_hat[i]) for i in range(len(y))))        

    def train(self, inputs, expected,epochs):
        for epoch in range(epochs):
             print(f"epoch={epoch}")
             for i, input in enumerate(inputs):
                  y_hats = self.feedfordward(input)
                  self.backpropagation(expected[i])
                  print(
                       f"loss round:{self.loss([y for y in y_hats.states],expected[i])}"
                  )

        

# Forward Propagation

![RNN-Froward-Propagation](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*55_hCfn3LiHj3MA1Gi4NJA.png)

# Backward Propagation


![RNN Architecture](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*RK3AlMRDGslZos_SG1LjzA.png)




In [8]:
if __name__ == "__main__":
  inputs = np.array([
      ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"],
      ["Z","Y","X","W","V","U","T","S","R","Q","P","O","N","M","L","K","J","I","H","G","F","E","D","C","B","A"],
      ["B","D","F","H","J","L","N","P","R","T","V","X","Z","A","C","E","G","I","K","M","O","Q","S","U","W","Y"],
      ["M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A","B","C","D","E","F","G","H","I","J","K","L"],
      ["H","G","F","E","D","C","B","A","L","K","J","I","P","O","N","M","U","T","S","R","Q","X","W","V","Z","Y"]
  ])

  expected = np.array([
      ["B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A"],
      ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"],
      ["C","E","G","I","K","M","O","Q","S","U","W","Y","A","B","D","F","H","J","L","N","P","R","T","V","X","Z"], 
      ["N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A","B","C","D","E","F","G","H","I","J","K","L","M"],
      ["I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","A","B","C","D","E","F","G","H"]
  ])

  onehot_inputs = string_to_one_hot(inputs)
  onehot_expected = string_to_one_hot(expected)

  rnn = MyRNN(vocab_size=len(string.ascii_uppercase),hidden_size=128,learning_rate=0.001)
  rnn.train(onehot_inputs,onehot_expected,epochs=1000)
  new_inputs = np.array([["B", "C", "D"]])
  for input in string_to_one_hot(new_inputs):
      predictions = rnn.feedfordward(input)
      output = np.argmax(predictions.states[-1])
      print(output) # index of the one-hot value of prediction
      print(f"The next letter is {string.ascii_uppercase[output]}") # mapping one hot to character

epoch=0
loss round:[174.63499269]
loss round:[132.58021654]
loss round:[119.89389737]
loss round:[115.57081989]
loss round:[112.15463836]
epoch=1
loss round:[109.59986208]
loss round:[109.25802499]
loss round:[110.60298101]
loss round:[110.36218613]
loss round:[122.20355476]
epoch=2
loss round:[132.810273]
loss round:[143.95564195]
loss round:[154.81958457]
loss round:[148.8558737]
loss round:[156.68732622]
epoch=3
loss round:[148.17927255]
loss round:[139.15122533]
loss round:[136.52682223]
loss round:[125.65894162]
loss round:[143.98833019]
epoch=4
loss round:[152.62516816]
loss round:[159.62815802]
loss round:[166.11340372]
loss round:[155.97457905]
loss round:[170.47985078]
epoch=5
loss round:[168.07642561]
loss round:[162.77693399]
loss round:[160.89839117]
loss round:[151.05970879]
loss round:[166.32067502]
epoch=6
loss round:[161.05862849]
loss round:[154.9883804]
loss round:[153.36329606]
loss round:[137.12585005]
loss round:[155.72507077]
epoch=7
loss round:[158.18844927]
loss

C:\Users\Hp\AppData\Local\Temp\ipykernel_10516\259731028.py:35: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return sum(-np.sum(y[i] * np.log(y_hat[i]) for i in range(len(y))))


loss round:[136.78947237]
loss round:[126.99260207]
loss round:[130.31506375]
epoch=11
loss round:[122.77692592]
loss round:[113.5820255]
loss round:[111.70449792]
loss round:[114.98777603]
loss round:[114.79930972]
epoch=12
loss round:[111.78451841]
loss round:[106.78814916]
loss round:[105.71357022]
loss round:[151.69484477]
loss round:[149.32382332]
epoch=13
loss round:[117.31936052]
loss round:[122.7118666]
loss round:[125.04859267]
loss round:[122.27367338]
loss round:[124.08419279]
epoch=14
loss round:[129.32454489]
loss round:[131.27580303]
loss round:[132.32842835]
loss round:[127.9351792]
loss round:[137.38564819]
epoch=15
loss round:[140.48317588]
loss round:[142.01029913]
loss round:[145.63051817]
loss round:[138.95117025]
loss round:[148.33495108]
epoch=16
loss round:[141.46296526]
loss round:[135.11353967]
loss round:[138.41151252]
loss round:[132.35191904]
loss round:[149.37743759]
epoch=17
loss round:[149.23121565]
loss round:[150.63589368]
loss round:[156.3385218]
loss 